I'm speculating if I can do my RR pairs by splitting the list of randoms into N parts and doing all the (N+1, 2) = (N+1)*N/2

In [1]:
from halotools.mock_observables.pair_counters import npairs_jackknife_3d
from halotools.mock_observables.catalog_analysis_helpers import cuboid_subvolume_labels
import yaml
import numpy as np

In [2]:
config_fname = '/home/users/swmclau2/Git/pearce/bin/trainer/xi_cosmo_trainer.yaml'

In [3]:
with open(config_fname, 'r') as ymlfile:
    cfg = yaml.load(ymlfile)

In [4]:
min_ptcl = int(cfg['HOD']['min_ptcl'])
r_bins = np.array(cfg['observation']['bins'] ).astype(float)

In [5]:
def compute_RR(cat, rbins, n_rands= 5, n_sub = 5, n_cores = 16):

    n_cores = cat._check_cores(n_cores)
#pos_m = return_xyz_formatted_array(x_m, y_m, z_m, period=cat.Lbox)
    rbins = np.array(rbins)
    np.random.seed(0)

    randoms = np.random.random((int(cat.halocat.ptcl_table['x'].shape[0] * n_rands), 3)) * cat.Lbox/cat.h  # Solution to NaNs: Just fuck me up with randoms
    
    j_index_randoms, N_sub_vol = cuboid_subvolume_labels(randoms, n_sub, cat.Lbox/cat.h)

    RR = npairs_jackknife_3d(randoms, randoms, rbins, period=cat.Lbox/cat.h,\
                jtags1=j_index_randoms, jtags2=j_index_randoms,\
                            N_samples=N_sub_vol, num_threads=n_cores)
    RR = np.diff(RR, axis=1)

    return RR

In [ ]:
from pearce.mocks.kittens import TestBox
cat = TestBox(boxno = 0, realization = 0, system = 'sherlock')
cat.load(1.0, HOD = str('zheng07'), particles = True, downsample_factor = 1e-2)

In [ ]:
RR = compute_RR(cat, r_bins, n_rands = 0.1)

In [ ]:
RR2 = compute_RR_hack(cat, r_bins, n_rands = 0.1)

In [ ]:
print np.all(RR == RR2)